In [2]:
import os
import cv2
import numpy as np
import face_recognition
import matplotlib.pyplot as plt
import pickle

In [3]:
def extract_features(img):
    faces = detect_faces(img)
    if faces:
        face_features = face_recognition.face_encodings(img, faces)
        return face_features[0]
    else:
        return None

# Function to detect faces in an image using face_recognition
def detect_faces(img):
    return face_recognition.face_locations(img)


In [4]:
cwd = os.getcwd()
print(cwd)


c:\Users\Muhammad Mehdi\OneDrive - University of Engineering and Technology Taxila\UET, Taxila\Semester 8\Final Year Project -II (FYP-2)\Face Recogntion


In [5]:
# Read all images and extract features
folder_name = 'Images'
image_dir = f"{cwd}\{folder_name}"  # Replace with the path to your images directory
print(image_dir)
images = []
features = []
for file in os.listdir(image_dir):
    img_path = os.path.join(image_dir, file)
    print(img_path)
    img = cv2.imread(img_path)
    faces = detect_faces(img)
    for face_location in faces:
        top, right, bottom, left = face_location
        face = img[top:bottom, left:right]
        face_features = extract_features(face)
        if face_features:
            features.append(face_features[0])
            images.append((file, face_location))

# Convert features to numpy array
features = np.array(features)

c:\Users\Muhammad Mehdi\OneDrive - University of Engineering and Technology Taxila\UET, Taxila\Semester 8\Final Year Project -II (FYP-2)\Face Recogntion\Images
c:\Users\Muhammad Mehdi\OneDrive - University of Engineering and Technology Taxila\UET, Taxila\Semester 8\Final Year Project -II (FYP-2)\Face Recogntion\Images\167A1160-2.jpg


: 

In [ ]:
# Cluster faces using face_recognition's compare_faces
# Instead of clustering, we will use face_recognition's compare_faces to find matches
unique_labels = []
labeled_images = {}
label_counter = 0

for i, feature in enumerate(features):
    match = False
    for label in unique_labels:
        if face_recognition.compare_faces([label], feature, tolerance=0.6)[0]:
            if label_counter not in labeled_images:
                labeled_images[label_counter] = []  # Initialize list if key doesn't exist
            labeled_images[label_counter].append(images[i])
            match = True
            break
    if not match:
        unique_labels.append(feature)
        labeled_images[label_counter] = [images[i]]
        label_counter += 1


In [ ]:
# Save labeled images information
with open('labeled_images.pkl', 'wb') as f:
    pickle.dump(labeled_images, f)
    

In [ ]:
# Function to find matches in provided image
def find_matches(query_image_path):
    # Read query image and detect faces
    query_image = cv2.imread(query_image_path)
    query_faces = detect_faces(query_image)
    
    # Iterate over each detected face in the query image
    for face_location in query_faces:
        top, right, bottom, left = face_location
        query_face = query_image[top:bottom, left:right]
        query_face_features = extract_features(query_face)
        
        # Compare the query face with stored face encodings
        if query_face_features:
            query_face_feature = query_face_features[0]
            for label, image_data in labeled_images.items():
                label_encoding = label
                for (file, bbox) in image_data:
                    img = cv2.imread(os.path.join(image_dir, file))
                    top, right, bottom, left = bbox
                    face = img[top:bottom, left:right]
                    face_features = extract_features(face)
                    if face_features:
                        face_feature = face_features[0]
                        # Compare the face features
                        if face_recognition.compare_faces([label_encoding], query_face_feature, tolerance=0.6)[0]:
                            # Display the image with annotated bounding box
                            cv2.rectangle(img, (left, top), (right, bottom), (255, 0, 0), 2)
                            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                            plt.show()

# Provide the path to the query image
query_image_path = 'img1.jpg'
find_matches(query_image_path)

: 